In [37]:
!pip install sqlalchemy pandas pyodbc


In [38]:
import pandas as pd
from sqlalchemy import create_engine
import pyodbc
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_timestamp, year, month, udf,to_date, when
from pyspark.sql.types import DoubleType, StringType


In [39]:
conn=pyodbc.connect('Driver={SQL Server};'
  'Server=DESKTOP-T08SBLB\SQLEXPRESS;'
  'Database=LAPD;'
  'Trusted_Connection=yes;')



In [40]:
if conn:
    print('Connection Successful')
else:
    print('Connection Failed')

Connection Successful


In [41]:
query='''SELECT
    TABLE_SCHEMA,
    TABLE_NAME,
    COLUMN_NAME,
    DATA_TYPE,
    IS_NULLABLE,
    CHARACTER_MAXIMUM_LENGTH
FROM
    INFORMATION_SCHEMA.COLUMNS
WHERE
    TABLE_CATALOG = 'LAPD'
ORDER BY
    TABLE_SCHEMA,
    TABLE_NAME,
    ORDINAL_POSITION;
'''

In [42]:
cursor = conn.cursor()
cursor.execute(query)
for row in cursor:
    print(row)

('dbo', 'Arrest_Data_from_2010_to_2019', 'Report ID', 'varchar', 'YES', 50)
('dbo', 'Arrest_Data_from_2010_to_2019', 'Report Type', 'varchar', 'YES', 50)
('dbo', 'Arrest_Data_from_2010_to_2019', 'Arrest Date', 'varchar', 'YES', 50)
('dbo', 'Arrest_Data_from_2010_to_2019', 'Time', 'varchar', 'YES', 50)
('dbo', 'Arrest_Data_from_2010_to_2019', 'Area ID', 'varchar', 'YES', 50)
('dbo', 'Arrest_Data_from_2010_to_2019', 'Area Name', 'varchar', 'YES', 50)
('dbo', 'Arrest_Data_from_2010_to_2019', 'Reporting District', 'varchar', 'YES', 50)
('dbo', 'Arrest_Data_from_2010_to_2019', 'Age', 'varchar', 'YES', 50)
('dbo', 'Arrest_Data_from_2010_to_2019', 'Sex Code', 'varchar', 'YES', 50)
('dbo', 'Arrest_Data_from_2010_to_2019', 'Descent Code', 'varchar', 'YES', 50)
('dbo', 'Arrest_Data_from_2010_to_2019', 'Charge Group Code', 'varchar', 'YES', 50)
('dbo', 'Arrest_Data_from_2010_to_2019', 'Charge Group Description', 'varchar', 'YES', 50)
('dbo', 'Arrest_Data_from_2010_to_2019', 'Arrest Type Code', 'v

In [43]:
# Define queries
query_1 = 'SELECT * FROM [dbo].[Arrest_Data_from_2010_to_2019];'
query_2 = 'SELECT * FROM [dbo].[Arrest_Data_from_2020_to_Present];'

In [44]:
# Execute query_1 and store results in a DataFrame
df_2010_to_2019 = pd.read_sql_query(query_1, conn)

C:\Users\16395\AppData\Local\Temp\ipykernel_15196\2029849224.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_2010_to_2019 = pd.read_sql_query(query_1, conn)


In [45]:
# Display sample data from each DataFrame
print("Sample data from Arrest_Data_from_2010_to_2019:")
print(df_2010_to_2019.head())

Sample data from Arrest_Data_from_2010_to_2019:
   Report ID Report Type Arrest Date  Time Area ID  Area Name  \
0  100629383         RFC  12/09/2010  1945      06  Hollywood   
1  100629384         RFC  12/09/2010  1845      06  Hollywood   
2  100629386         RFC  12/09/2010  1845      06  Hollywood   
3  100629387         RFC  12/09/2010  2355      06  Hollywood   
4  100629388         RFC  12/09/2010  2335      06  Hollywood   

  Reporting District Age Sex Code Descent Code  ...  \
0               0647  20        M            H  ...   
1               0648  20        M            H  ...   
2               0648  18        M            H  ...   
3               0665  21        F            H  ...   
4               0637  19        M            H  ...   

       Disposition Description                                  Address  \
0  MISDEMEANOR COMPLAINT FILED          GORDON                       ST   
1  MISDEMEANOR COMPLAINT FILED          CARLTON                      WY   
2  MI

In [46]:
# Execute query_2 and store results in a DataFrame
df_2020_to_present = pd.read_sql_query(query_2, conn)

C:\Users\16395\AppData\Local\Temp\ipykernel_15196\1424259808.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_2020_to_present = pd.read_sql_query(query_2, conn)


In [47]:
print("\nSample data from Arrest_Data_from_2020_to_Present:")
print(df_2020_to_present.head())


Sample data from Arrest_Data_from_2020_to_Present:
  Report ID Report Type             Arrest Date  Time Area ID    Area Name  \
0   6636966     BOOKING  07/06/2023 12:00:00 AM  2250      08      West LA   
1   6637119     BOOKING  07/07/2023 12:00:00 AM  1000      03    Southwest   
2   6624479     BOOKING  06/15/2023 12:00:00 AM  1850      07     Wilshire   
3   6636128     BOOKING  07/05/2023 12:00:00 AM  1550      02      Rampart   
4   6636650     BOOKING  07/06/2023 12:00:00 AM  1335      12  77th Street   

  Reporting District Age Sex Code Descent Code  ...  \
0               0817  46        M            B  ...   
1               0396  39        M            B  ...   
2               0724  33        F            H  ...   
3               0218  30        F            B  ...   
4               1258  31        M            H  ...   

       Disposition Description                                  Address  \
0                  NOT FIREARM                                   W/GBI"  

In [48]:
# Increase Spark memory limit
spark = SparkSession.builder \
    .appName("Pandas to PySpark") \
    .config("spark.driver.memory", "4g") \
    .config("spark.sql.legacy.timeParserPolicy", "LEGACY") \
    .getOrCreate()

In [49]:
# Convert the pandas DataFrame to a PySpark DataFrame
spark_df_2010_to_2019 = spark.createDataFrame(df_2010_to_2019)

In [50]:
# Convert the pandas DataFrame to a PySpark DataFrame for df1
spark_df_2020_to_present = spark.createDataFrame(df_2020_to_present)

In [51]:
# Union the PySpark DataFrames
combined_arrest_data = spark_df_2010_to_2019.union(spark_df_2020_to_present)

In [52]:
combined_arrest_data.show(5)

+---------+-----------+-----------+----+-------+---------+------------------+---+--------+------------+-----------------+------------------------+----------------+----------+------------------+-----------------------+--------------------+--------------------+-------+---------+--------------------+------------+------------+----------------+---------------------+
|Report ID|Report Type|Arrest Date|Time|Area ID|Area Name|Reporting District|Age|Sex Code|Descent Code|Charge Group Code|Charge Group Description|Arrest Type Code|    Charge|Charge Description|Disposition Description|             Address|        Cross Street|    LAT|      LON|            Location|Booking Date|Booking Time|Booking Location|Booking Location Code|
+---------+-----------+-----------+----+-------+---------+------------------+---+--------+------------+-----------------+------------------------+----------------+----------+------------------+-----------------------+--------------------+--------------------+-------+-----

In [53]:
combined_arrest_data = combined_arrest_data.filter(col('Arrest Date').isNotNull())

In [54]:
# Extract Latitude and Longitude from 'Location'
def extract_latitude(location):
    try:
        return float(location.split()[1][1:])
    except Exception as e:
        print(f"Error extracting latitude: {e}")
        return None

def extract_longitude(location):
    try:
        return float(location.split()[2][:-1])
    except Exception as e:
        print(f"Error extracting longitude: {e}")
        return None

In [55]:
latitude_udf = udf(extract_latitude, DoubleType())
longitude_udf = udf(extract_longitude, DoubleType())

combined_arrest_data = combined_arrest_data.withColumn('Latitude', latitude_udf(col('Location')))
combined_arrest_data = combined_arrest_data.withColumn('Longitude', longitude_udf(col('Location')))

In [56]:
# Function to clean address
def clean_address(address):
    if address:
        return ' '.join(address.split())
    return address

clean_address_udf = udf(clean_address, StringType())

In [57]:
# Apply the cleaning function to the Address and Cross Street columns
combined_arrest_data = combined_arrest_data.withColumn('Address', clean_address_udf(col('Address')))
combined_arrest_data = combined_arrest_data.withColumn('Cross Street', clean_address_udf(col('Cross Street')))

In [58]:
# Drop specified columns
columns_to_drop = [
    'Descent Code', 'Charge Group Code', 'Charge Group Description',
    'Disposition Description', 'Booking Date', 'Booking Time',
    'Booking Location', 'Booking Location Code'
]
combined_arrest_data = combined_arrest_data.drop(*columns_to_drop)

In [59]:
# Fill missing values in 'Charge Description, cross street'
combined_arrest_data = combined_arrest_data.withColumn('Charge Description', when(col('Charge Description').isNull(), 'Unknown').otherwise(col('Charge Description')))

In [60]:
combined_arrest_data = combined_arrest_data.withColumn('Cross Street', when(col('Cross Street').isNull(), 'NAN').otherwise(col('Cross Street')))
# Fill missing values with an empty string
combined_arrest_data = combined_arrest_data.na.fill('NAN')

In [61]:
# Show some results to verify final DataFrame
combined_arrest_data.show(5)

+---------+-----------+-----------+----+-------+---------+------------------+---+--------+----------------+----------+------------------+------------+------------+-------+---------+--------------------+---------+---------+
|Report ID|Report Type|Arrest Date|Time|Area ID|Area Name|Reporting District|Age|Sex Code|Arrest Type Code|    Charge|Charge Description|     Address|Cross Street|    LAT|      LON|            Location| Latitude|Longitude|
+---------+-----------+-----------+----+-------+---------+------------------+---+--------+----------------+----------+------------------+------------+------------+-------+---------+--------------------+---------+---------+
|100629383|        RFC| 12/09/2010|1945|     06|Hollywood|              0647| 20|       M|               M|41.27CLAMC|DRINKING IN PUBLIC|   GORDON ST|   SUNSET BL|34.0981|-118.3196|POINT (-118.3196 ...|-118.3196|  34.0981|
|100629384|        RFC| 12/09/2010|1845|     06|Hollywood|              0648| 20|       M|               M|4

In [62]:
from pyspark.sql.functions import regexp_replace
# Transform the 'Date Rptd' column to remove the time part
combined_arrest_data = combined_arrest_data.withColumn(
    "Arrest Date",
    regexp_replace("Arrest Date", r"\s\d{2}:\d{2}:\d{2}\s[APM]{2}", "")
)


In [66]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, regexp_replace

# Assuming you have already created your SparkSession and loaded your DataFrame 'df'

# Define patterns to replace
patterns_to_replace = r'["\\/]'

# Apply regexp_replace to filter out rows where any column value contains the specified patterns
filtered_df = combined_arrest_data.filter(
    ~col("Charge").rlike(patterns_to_replace)
    & ~col("Charge Description").rlike(patterns_to_replace)
    & ~col("Address").rlike(patterns_to_replace)
    & ~col("Cross Street").rlike(patterns_to_replace)
    & ~col("LAT").rlike(patterns_to_replace)
    & ~col("LON").rlike(patterns_to_replace)
    & ~col("Location").rlike(patterns_to_replace)
    & ~col("Latitude").rlike(patterns_to_replace)
    & ~col("Longitude").rlike(patterns_to_replace)
    # Add more columns as needed
)

# Show the filtered DataFrame
filtered_df.show()


+---------+-----------+-----------+----+-------+---------+------------------+---+--------+----------------+-----------+------------------+-----------------+------------+-------+---------+--------------------+---------+---------+
|Report ID|Report Type|Arrest Date|Time|Area ID|Area Name|Reporting District|Age|Sex Code|Arrest Type Code|     Charge|Charge Description|          Address|Cross Street|    LAT|      LON|            Location| Latitude|Longitude|
+---------+-----------+-----------+----+-------+---------+------------------+---+--------+----------------+-----------+------------------+-----------------+------------+-------+---------+--------------------+---------+---------+
|100629383|        RFC| 12/09/2010|1945|     06|Hollywood|              0647| 20|       M|               M| 41.27CLAMC|DRINKING IN PUBLIC|        GORDON ST|   SUNSET BL|34.0981|-118.3196|POINT (-118.3196 ...|-118.3196|  34.0981|
|100629384|        RFC| 12/09/2010|1845|     06|Hollywood|              0648| 20|   

In [68]:
# Filter out rows where all columns are null or empty
filtered_df = filtered_df.filter(
    ~(col("Report ID").isNull() | col("Charge").eqNullSafe(""))
    & ~(col("Report Type").isNull() | col("Charge Description").eqNullSafe(""))
    & ~(col("Arrest Date").isNull() | col("Charge Description").eqNullSafe(""))
    & ~(col("Time").isNull() | col("Charge Description").eqNullSafe(""))
    & ~(col("Area ID").isNull() | col("Charge Description").eqNullSafe(""))
    & ~(col("Area Name").isNull() | col("Charge Description").eqNullSafe(""))
    & ~(col("Reporting District").isNull() | col("Charge Description").eqNullSafe(""))
    & ~(col("Age").isNull() | col("Charge Description").eqNullSafe(""))
    & ~(col("Sex Code").isNull() | col("Charge Description").eqNullSafe(""))
    & ~(col("Arrest Type Code").isNull() | col("Charge Description").eqNullSafe(""))
    & ~(col("Charge").isNull() | col("Charge Description").eqNullSafe(""))
    & ~(col("Charge Description").isNull() | col("Charge Description").eqNullSafe(""))
    & ~(col("Address").isNull() | col("Address").eqNullSafe(""))
    & ~(col("Cross Street").isNull() | col("Cross Street").eqNullSafe(""))
    & ~(col("LAT").isNull() | col("LAT").eqNullSafe(""))
    & ~(col("LON").isNull() | col("LON").eqNullSafe(""))
    & ~(col("Location").isNull() | col("Location").eqNullSafe(""))
    & ~(col("Latitude").isNull() | col("Latitude").eqNullSafe(""))
    & ~(col("Longitude").isNull() | col("Longitude").eqNullSafe(""))
    # Add more columns as needed
)
# Show the filtered DataFrame
filtered_df.show()

+---------+-----------+-----------+----+-------+---------+------------------+---+--------+----------------+----------+------------------+------------+---------------+-------+---------+--------------------+---------+---------+
|Report ID|Report Type|Arrest Date|Time|Area ID|Area Name|Reporting District|Age|Sex Code|Arrest Type Code|    Charge|Charge Description|     Address|   Cross Street|    LAT|      LON|            Location| Latitude|Longitude|
+---------+-----------+-----------+----+-------+---------+------------------+---+--------+----------------+----------+------------------+------------+---------------+-------+---------+--------------------+---------+---------+
|100629383|        RFC| 12/09/2010|1945|     06|Hollywood|              0647| 20|       M|               M|41.27CLAMC|DRINKING IN PUBLIC|   GORDON ST|      SUNSET BL|34.0981|-118.3196|POINT (-118.3196 ...|-118.3196|  34.0981|
|100629384|        RFC| 12/09/2010|1845|     06|Hollywood|              0648| 20|       M|      

In [71]:
# Define the output path where you want to save the CSV file locally
output_path = r"C:\Users\16395\Downloads\final project\bigdatasource\LAPD\sqlserver_pyspark_transformation\output.csv"

In [72]:
# Reduce the number of partitions to 1 to create a single CSV file
combined_arrest_data = combined_arrest_data.coalesce(1)

In [73]:
# Save the DataFrame to a single CSV file on your local machine
combined_arrest_data.write.csv(output_path, mode="overwrite", header=True)

In [ ]:
combined_arrest